<a href="https://colab.research.google.com/github/shubhangkhare/Learnings/blob/main/UP20%20Analytics%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries

In [1]:
!pip install mplfinance yfinance -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install fuzzywuzzy python-Levenshtein -q

In [2]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import requests
pd.set_option('display.max_columns', None)

# Plot Candlesticks

In [133]:
# Retrieve AAPL historical data
symbol = 'DAAWAT.NS' # Share symbol + NSE suffix
ticker = yf.Ticker(symbol)
data = ticker.history(period="1d")

# Create candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

# Customize the chart layout
fig.update_layout(title=f"{symbol} Candlestick Chart (1 Month)",
                  yaxis_title="Price",
                  xaxis_rangeslider_visible=False)

# Display the chart
fig.show()

ERROR:yfinance:$DAAWAT.NS: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


# UP 20 - Code

## Reset index

In [4]:
data.reset_index(inplace = True)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-10-08 00:00:00+05:30,624.900024,645.900024,613.200012,640.599976,2350816,0.0,0.0
1,2024-10-09 00:00:00+05:30,646.400024,649.000000,631.549988,634.299988,1270367,0.0,0.0
2,2024-10-10 00:00:00+05:30,641.950012,648.400024,631.049988,645.400024,1649770,0.0,0.0
3,2024-10-11 00:00:00+05:30,645.000000,645.400024,637.450012,642.299988,1109827,0.0,0.0
4,2024-10-14 00:00:00+05:30,642.500000,645.700012,631.000000,634.450012,1811001,0.0,0.0


## Get Candle Color

In [5]:
def green_candles(open, close):
    if close > open:
        return 'green'
    else:
        return 'red'

data['Candle Color'] = data.apply(lambda x: green_candles(x['Open'], x['Close']), axis = 1)
data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Candle Color
0,2024-10-08 00:00:00+05:30,624.900024,645.900024,613.200012,640.599976,2350816,0.0,0.0,green
1,2024-10-09 00:00:00+05:30,646.400024,649.000000,631.549988,634.299988,1270367,0.0,0.0,red
2,2024-10-10 00:00:00+05:30,641.950012,648.400024,631.049988,645.400024,1649770,0.0,0.0,green
3,2024-10-11 00:00:00+05:30,645.000000,645.400024,637.450012,642.299988,1109827,0.0,0.0,red
4,2024-10-14 00:00:00+05:30,642.500000,645.700012,631.000000,634.450012,1811001,0.0,0.0,red


## Get UP 20 Dates

In [6]:
def check_up_20_helper(df_temp, threshold):
    """
    Check if the maximum price increase in a given DataFrame exceeds a specified threshold.

    Parameters:
    df_temp (pd.DataFrame): DataFrame containing stock market data with columns 'Date', 'High', 'Low', and 'Candle Color'.
    threshold (float): Percentage threshold to check against.

    Returns:
    pd.DataFrame or None: DataFrame with the start date, end date, and percentage increase if the condition is met, otherwise None.
    """

    color_pattern = tuple(df_temp['Candle Color'].values)

    if color_pattern[0] != 'green':
        return None

    if color_pattern == ('green', 'green', 'green'):
        pass
    elif color_pattern == ('green', 'green', 'red'):
        df_temp = df_temp.iloc[0:2, :]
    elif color_pattern in [('green', 'red', 'red'), ('green', 'red', 'green')]:
        df_temp = df_temp.iloc[0:1, :]  # Keep df_temp as a DataFrame with one row
    else:
        return None

    # Ensure df_temp is a DataFrame after slicing
    if isinstance(df_temp, pd.Series):
        df_temp = df_temp.to_frame().T

    # Calculate max and min prices
    max_price = df_temp['High'].max()
    min_price = df_temp['Low'].min()

    # Calculate percentage increase
    per_increase = 100 * (max_price - min_price) / min_price
    #print(per_increase)

    # Get start and end dates
    start_date = df_temp['Date'].iloc[0]  # Access the first element using .iloc[0]
    end_date = df_temp['Date'].iloc[-1]  # Access the last element using .iloc[-1]

    # Check if the percentage increase meets or exceeds the threshold
    if per_increase >= threshold:
        output_dict = {
            'Start Date': [start_date],
            'End Date': [end_date],
            '% Increase': [per_increase]
        }
        output_df = pd.DataFrame(output_dict)
        return output_df
    else:
        return None

#result = check_up_20_helper(df_temp, 5)
#result.head()

In [51]:
def check_up_20(symbol = "ADANIPOWER.NS", period="1mo", thres = 5):
    # Load Data
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period)
    # Reset Index
    data.reset_index(inplace = True)
    # Get Candle Color
    data['Candle Color'] = data.apply(lambda x: green_candles(x['Open'], x['Close']), axis = 1)

    result_list = []
    for idx in range(data.shape[0]):
        df_temp = data.iloc[idx:idx+3, :]
        #return df_temp
        output_df = check_up_20_helper(df_temp, thres)
        if output_df is not None:
            output_df['symbol'] = symbol
            result_list.append(output_df)
    if len(result_list) > 0:
        result = pd.concat(result_list)
        return result

In [81]:
# ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
result = check_up_20(symbol = "ADANIPOWER.NS", period="1Y", thres = 20)
result.head()

,Start Date,End Date,% Increase,symbol
0,2023-11-24 00:00:00+05:30,2023-11-28 00:00:00+05:30,22.114123,ADANIPOWER.NS
0,2023-12-05 00:00:00+05:30,2023-12-06 00:00:00+05:30,29.265354,ADANIPOWER.NS
0,2024-05-30 00:00:00+05:30,2024-06-03 00:00:00+05:30,32.610464,ADANIPOWER.NS
0,2024-05-31 00:00:00+05:30,2024-06-03 00:00:00+05:30,27.242382,ADANIPOWER.NS


# Stock Screener

# Load UP 20 Companies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [103]:
df_screen = pd.read_excel('/content/drive/MyDrive/Algo Trading/UP20.xlsx')
df_screen.head()

,Name,Sub-Sector,Market Cap,Close Price,PE Ratio,Net Income,ROCE,Debt to Equity
0,Tata Consultancy Services Ltd,IT Services & Consulting,1500420.89,4147.00,32.68,45908.00,63.17,0.09
1,Infosys Ltd,IT Services & Consulting,757951.03,1829.95,28.89,26233.00,37.50,0.09
2,ITC Ltd,FMCG - Tobacco,598025.84,478.05,29.23,20458.78,34.87,0.00
3,Hindustan Unilever Ltd,FMCG - Household Products,589207.00,2507.70,57.33,10277.00,21.73,0.03
4,HCL Technologies Ltd,IT Services & Consulting,497251.98,1837.50,31.67,15702.00,28.21,0.08


## Get NSE symbols

In [97]:
# Define the API URL
url = "https://financialmodelingprep.com/api/v3/stock/list?apikey=urY2vTrf7Lw87kNKvWV5DKeg3x2eEhRU"
# Fetch data from the API
response = requests.get(url)
data = response.json()

# Convert JSON response to DataFrame
df = pd.DataFrame(data)

df = df[df['exchangeShortName'] == 'NSE']
df['name'] = df['name'].str.replace('Limited', 'Ltd')

df.head()

,symbol,name,price,exchange,exchangeShortName,type
36837,JWL.NS,Jupiter Wagons Ltd,493.95,National Stock Exchange of India,NSE,stock
36838,JYOTICNC.NS,Jyoti CNC Automation Ltd,1130.50,National Stock Exchange of India,NSE,stock
36840,UTIAMC.NS,UTI Asset Management Company Ltd,1323.25,National Stock Exchange of India,NSE,stock
36841,IPL.NS,India Pesticides Ltd,199.58,National Stock Exchange of India,NSE,stock
36842,IRFC.NS,Indian Railway Finance Corporation Ltd,148.54,National Stock Exchange of India,NSE,stock


In [119]:
from fuzzywuzzy import process

# Function to apply fuzzy matching
def get_best_match(query, choices, threshold=98):
    if query in choices:
        return query
    match, score = process.extractOne(query, choices)
    return match if score >= threshold else None

# Create a list of names from the right DataFrame for fuzzy matching
name_choices = df['name'].tolist()

# Apply fuzzy matching to the 'Name' column in df_screen and create a new column with best matches
df_screen['fuzzy_match'] = df_screen['Name'].apply(lambda x: get_best_match(x, name_choices))

# Now merge on the fuzzy matches
df_1 = pd.merge(df_screen, df, how='inner', left_on='fuzzy_match', right_on='name')

# Show the result
df_1.head()

,Name,Sub-Sector,Market Cap,Close Price,PE Ratio,Net Income,ROCE,Debt to Equity,fuzzy_match,symbol,name,price,exchange,exchangeShortName,type
0,Tata Consultancy Services Ltd,IT Services & Consulting,1500420.89,4147.00,32.68,45908.00,63.17,0.09,Tata Consultancy Services Ltd,TCS.NS,Tata Consultancy Services Ltd,4147.00,National Stock Exchange of India,NSE,stock
1,Infosys Ltd,IT Services & Consulting,757951.03,1829.95,28.89,26233.00,37.50,0.09,Infosys Ltd,INFY.NS,Infosys Ltd,1829.95,National Stock Exchange of India,NSE,stock
2,ITC Ltd,FMCG - Tobacco,598025.84,478.05,29.23,20458.78,34.87,0.00,ITC Ltd,ITC.NS,ITC Ltd,478.05,National Stock Exchange of India,NSE,stock
3,Hindustan Unilever Ltd,FMCG - Household Products,589207.00,2507.70,57.33,10277.00,21.73,0.03,Hindustan Unilever Ltd,HINDUNILVR.NS,Hindustan Unilever Ltd,2507.70,National Stock Exchange of India,NSE,stock
4,HCL Technologies Ltd,IT Services & Consulting,497251.98,1837.50,31.67,15702.00,28.21,0.08,HCL Technologies Ltd,HCLTECH.NS,HCL Technologies Ltd,1837.50,National Stock Exchange of India,NSE,stock


In [128]:
df_1 = pd.merge(df_screen, df, how='inner', left_on = 'Name', right_on = 'name')
df_1.head()

,Name,Sub-Sector,Market Cap,Close Price,PE Ratio,Net Income,ROCE,Debt to Equity,fuzzy_match,symbol,name,price,exchange,exchangeShortName,type
0,Tata Consultancy Services Ltd,IT Services & Consulting,1500420.89,4147.00,32.68,45908.00,63.17,0.09,Tata Consultancy Services Ltd,TCS.NS,Tata Consultancy Services Ltd,4147.00,National Stock Exchange of India,NSE,stock
1,Infosys Ltd,IT Services & Consulting,757951.03,1829.95,28.89,26233.00,37.50,0.09,Infosys Ltd,INFY.NS,Infosys Ltd,1829.95,National Stock Exchange of India,NSE,stock
2,ITC Ltd,FMCG - Tobacco,598025.84,478.05,29.23,20458.78,34.87,0.00,ITC Ltd,ITC.NS,ITC Ltd,478.05,National Stock Exchange of India,NSE,stock
3,Hindustan Unilever Ltd,FMCG - Household Products,589207.00,2507.70,57.33,10277.00,21.73,0.03,Hindustan Unilever Ltd,HINDUNILVR.NS,Hindustan Unilever Ltd,2507.70,National Stock Exchange of India,NSE,stock
4,HCL Technologies Ltd,IT Services & Consulting,497251.98,1837.50,31.67,15702.00,28.21,0.08,HCL Technologies Ltd,HCLTECH.NS,HCL Technologies Ltd,1837.50,National Stock Exchange of India,NSE,stock


In [129]:
df_1.shape, df_screen.shape

((233, 15), (281, 9))

## Find UP-20 Oppurtunities

In [84]:
def get_all(df, period="1mo", thres = 5):
    symbols = df['symbol'].to_list()
    result_list = []

    for symbol in symbols:
        result = check_up_20(symbol = symbol, period = period, thres = thres)
        if result is not None:
            result_list.append(result)

    if len(result_list) > 0:
        result = pd.concat(result_list)
        return result

In [136]:
df_all = get_all(df_1, period="1mo", thres = 18.5)
df_all.head()

ERROR:yfinance:$MCDOWELL-N.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$CADILAHC.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$MINDAIND.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$CEBBCO.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$SUPPETRO.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$DAAWAT.NS: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
ERROR:yfinance:$KENNAMET.NS: possibly delisted; no price data found  (period=1mo)
ERROR:yfinance:$KOVAI.NS: possibly delisted; no price data found  (period=1mo)
ERROR:yfinanc

,Start Date,End Date,% Increase,symbol
0,2024-10-30 00:00:00+05:30,2024-11-01 00:00:00+05:30,22.777044,WAAREEENER.NS
0,2024-10-31 00:00:00+05:30,2024-11-04 00:00:00+05:30,23.520500,WAAREEENER.NS
0,2024-11-01 00:00:00+05:30,2024-11-05 00:00:00+05:30,31.592922,WAAREEENER.NS
0,2024-11-04 00:00:00+05:30,2024-11-06 00:00:00+05:30,38.016224,WAAREEENER.NS
0,2024-11-05 00:00:00+05:30,2024-11-06 00:00:00+05:30,27.690784,WAAREEENER.NS


In [137]:
df_all['symbol'].nunique()

20

In [138]:
df_all.head(50)

,Start Date,End Date,% Increase,symbol
0,2024-10-30 00:00:00+05:30,2024-11-01 00:00:00+05:30,22.777044,WAAREEENER.NS
0,2024-10-31 00:00:00+05:30,2024-11-04 00:00:00+05:30,23.520500,WAAREEENER.NS
0,2024-11-01 00:00:00+05:30,2024-11-05 00:00:00+05:30,31.592922,WAAREEENER.NS
0,2024-11-04 00:00:00+05:30,2024-11-06 00:00:00+05:30,38.016224,WAAREEENER.NS
0,2024-11-05 00:00:00+05:30,2024-11-06 00:00:00+05:30,27.690784,WAAREEENER.NS
0,2024-10-28 00:00:00+05:30,2024-10-30 00:00:00+05:30,21.676681,GILLETTE.NS
0,2024-10-30 00:00:00+05:30,2024-11-01 00:00:00+05:30,30.931352,POLYMED.NS
0,2024-10-31 00:00:00+05:30,2024-11-01 00:00:00+05:30,21.518533,POLYMED.NS
0,2024-10-08 00:00:00+05:30,2024-10-10 00:00:00+05:30,21.059398,JAIBALAJI.NS
0,2024-10-08 00:00:00+05:30,2024-10-09 00:00:00+05:30,25.897042,AKZOINDIA.NS
